# Homework 3

In [2]:
import gzip
from collections import defaultdict
import random

def readCSV(path):
  f = gzip.open(path, 'rt')
  f.readline()
  for l in f:
    yield l.strip().split(',')
    
data = []
for l in readCSV("train_Interactions.csv.gz"):
    data.append(l)
train_data=data[:190000]
valid_data=data[190000:]

#### 1. Although we have built a validation set, it only consists of positive samples. For this task we also need examples of user/item pairs that weren’t read. For each entry (user,book) in the validation set, sample a negative entry by randomly choosing a book that user hasn’t read.

In [3]:
unique_user=set(l[0] for l in data)
unique_book=set(l[1] for l in data)

In [4]:
userbooks = defaultdict(set)
bookusers = defaultdict(set)

for user,book,r in readCSV("train_Interactions.csv.gz"):
  userbooks[user].add(book)
  bookusers[book].add(user)

In [5]:
import random
valid_read=[]
n=0
while n<10000:
    for user, book, rating in valid_data:
        read=userbooks[user]
        unread=unique_book-read
        valid_read.append((user, random.choice(list(unread)), 0))
        n+=1

In [6]:
train_read=[]
for user, book, r in valid_data:
    valid_read.append((user, book, 1))
for user, book, r in train_data:
    train_read.append((user, book, 1)) 

#### 1. Evaluate the performance (accuracy) of the baseline model on the validation set you have built (1 mark).

In [7]:
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
  bookCount[book] += 1
  totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalRead/2: break

In [8]:
valid_read_prediction=[]
for u, b, _ in valid_read:
    if b in return1:
        valid_read_prediction.append((u, b, 1))
    else:
        valid_read_prediction.append((u, b, 0))

In [9]:
correct=0
for i in range(0,len(valid_read_prediction)):
    if valid_read_prediction[i][2] == valid_read[i][2]:
        correct+=1
accuracy = correct / len(valid_read)
accuracy

0.6557

#### 2. The existing ‘read prediction’ baseline just returns True if the item in question is ‘popular,’ using a threshold of the 50th percentile of popularity (totalRead/2). Assuming that the ‘non-read’ test examples are a random sample of user-book pairs, this threshold may not be the best one. See if you can find a better threshold and report its performance on your validation set (1 mark).

In [10]:
import numpy as np
thresholds=np.arange(0.01, 1, 0.01)

In [ ]:
accuracies=[]
def thresholdfinder(threshold):
    return1 = set()
    count = 0
    for ic, i in mostPopular:
      count += ic
      return1.add(i)
      if count > totalRead*threshold: break
    valid_read_prediction=[]
    for u, b, _ in valid_read:
        if b in return1:
            valid_read_prediction.append((u, b, 1))
        else:
            valid_read_prediction.append((u, b, 0))

    correct=0
    for i in range(0,len(valid_read_prediction)):
        if valid_read_prediction[i][2] == valid_read[i][2]:
            correct+=1

    accuracy = correct / len(valid_read)
    accuracies.append(accuracy)

In [ ]:
for i in range(0,len(thresholds)):
     thresholdfinder(thresholds[i])

In [ ]:
import pandas as pd
df = pd.DataFrame(list(zip(thresholds, accuracies)), columns =['threshold', 'accuracy'])
import matplotlib.pyplot as plt 
df.plot(kind='line',x='threshold',y='accuracy')
plt.show()
threshold=thresholds[df['accuracy']==max(df['accuracy'])][0]
df[df['accuracy']==max(df['accuracy'])]

In [ ]:
print('A threshold at the '+ str(int(threshold*100)) +'th percentile of popularity (TotalRead * ' + str(threshold) + ') achieves highest accuracy of ' + str(max(df['accuracy']))+' on the validation set.')

#### 3. A stronger baseline than the one provided might make use of the Jaccard similarity (or another similarity metric). Given a pair (u, b) in the validation set, consider all training items b' that user u has read. For each, compute the Jaccard similarity between b and b', i.e., users (in the training set) who have read b and users who have read b'. Predict as ‘read’ if the maximum of these Jaccard similarities exceeds athreshold (you may choose the threshold that works best). Report the performance on your validation set (1 mark).

In [11]:
userbooks_train = defaultdict(set)
bookusers_train = defaultdict(set)

for user,book,r in train_read:
    userbooks_train[user].add(book)
    bookusers_train[book].add(user)

In [12]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer / denom

In [13]:
def mostSimilar(user, book):
    similarities = []
    users = bookusers_train[book] # users who have read the query book
    books = userbooks_train[user] # books the query user has read          
    for readbook in books: # for every book the query user has read, find its user
        similarusers=bookusers_train[readbook] # find similar users who has read one of the book read by the user
        sim = Jaccard(users, similarusers)
        similarities.append(sim)
        similarities.sort(reverse=True)  
    return similarities[0]

In [14]:
allsimilarities=[]
for user,book,r in valid_read:
    allsimilarities.append(mostSimilar(user, book))
max(allsimilarities)

0.10344827586206896

In [15]:
min(allsimilarities)

0.0

In [16]:
def jaccard_thresholdfinder(threshold):
    jaccard_predictions=[]
    for user,book,r in valid_read:
        s = mostSimilar(user, book)
        if s > threshold:
            jaccard_predictions.append((user, book, 1))
        else:
            jaccard_predictions.append((user, book, 0))

    correct=0
    for i in range(0,len(valid_read_prediction)):
        if jaccard_predictions[i][2] == valid_read[i][2]:
                correct+=1
    accuracy = correct / len(valid_read)
    accuracies.append(accuracy)

In [ ]:
accuracies=[]
thresholds_jaccard=np.arange(0, 0.1, 0.002)
for i in range(0,len(thresholds_jaccard)):
     jaccard_thresholdfinder(thresholds_jaccard[i])

In [ ]:
df_jaccard = pd.DataFrame(list(zip(thresholds_jaccard, accuracies)), columns =['threshold', 'accuracy'])
df_jaccard[df_jaccard['accuracy']==max(df_jaccard['accuracy'])]
df_jaccard.plot(kind='line',x='threshold',y='accuracy')
plt.show()

In [ ]:
threshold_jaccard=thresholds_jaccard[df_jaccard['accuracy']==max(df_jaccard['accuracy'])][0]

In [ ]:
print('A threshold at the '+ str(threshold_jaccard) +' achieves highest accuracy of ' + str(max(df_jaccard['accuracy']))+' on the validation set.')

#### 4. Improve the above predictor by incorporating both a Jaccard-based threshold and a popularity based threshold. Report the performance on your validation set (1 mark).

# two threshold finders

In [17]:
similarities_valid=defaultdict(int)
for u, b, _ in valid_read:
    s = mostSimilar(u, b)
    similarities_valid[u]+=(s)

In [18]:
accuracies=[]
def thresholdfinders(t1,t2):
    prediction_4=[]
    for u, b, _ in valid_read:
        return1 = set()
        count = 0
        for ic, i in mostPopular:
          count += ic
          return1.add(i)
          if count > totalRead*t1: break
        if b in return1 and similarities_valid[u] > t2:
            prediction_4.append((u, b, 1))
        else:
            prediction_4.append((u, b, 0))

    correct=0
    for i in range(0,len(prediction_4)):
        if prediction_4[i][2] == valid_read[i][2]:
                correct+=1
    accuracy = correct / len(valid_read)
    accuracies.append(accuracy)

In [19]:
boththreshold=[]
t1=np.arange(0.5843, 0.5849, 0.0002)
t2=np.arange(0, 0.003, 0.002)

I tried big ranges at first and then narrowed down this small range. To save running time, I only included this last range.

In [ ]:
accuracies=[]
for a in range(0,len(t1)):
    for b in range(0,len(t2)):
        thresholdfinders(t1[a],t2[b])
        boththreshold.append((t1[a],t2[b]))

In [ ]:
df_both = pd.DataFrame(list(zip(boththreshold, accuracies)), columns =['popularity & similarity threshold', 'accuracy'])
df_both.plot(kind='line', y='accuracy')
plt.show()
df_both

In [20]:
return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead*0.5847: break

In [21]:
def Cosine(s1, s2):
    numer = len(s1.intersection(s2))   
    denom = len(s1)*len(s2)
    return numer / denom

In [22]:
def mostCosine(user, book):
    similarities = []
    users = bookusers[book] # users who have read the query book
    books = userbooks[user] # books the query user has read          
    for readbook in books: # for every book the query user has read, find its user
        if not readbook == book:
            similarusers=bookusers[readbook] # find similar users who has read one of the book read by the user
            sim = Cosine(users, similarusers)
            similarities.append(sim)
            similarities.sort(reverse=True)  
        else:
            similarities.append(0)
    return similarities[0]

In [26]:
predictions = open("predictions_Read.txt", 'w')
for l in open("pairs_Read.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,b = l.strip().split('-')
    c = mostCosine(str(u), str(b))
    if c > 0.0016:
        predictions.write(u + '-' + b + ",1\n")
    else:
        predictions.write(u + '-' + b + ",0\n")
predictions.close()

In [ ]:
predictions = open("predictions_Read.txt", 'w')
for l in open("pairs_Read.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,b = l.strip().split('-')
    j = mostSimilar(str(u), str(b))
    c = mostCosine(str(u), str(b))
    if b in return1 and j > 0.002 and c > 0.0016:
        predictions.write(u + '-' + b + ",1\n")
    else:
        predictions.write(u + '-' + b + ",0\n")
predictions.close()

#### 5. To run our model on the test set, we’ll have to use the files ‘pairs Read.txt’ to find the reviewerID/itemID pairs about which we have to make predictions. Using that data, run the above model and upload your solution to Kaggle. Tell us your Kaggle user name (1 mark). If you’ve already uploaded a better solution to Kaggle, that’s fine too!

Chengsheng Wu

#### 9. Fit a predictor of the form rating(user, item) = α + βuser + βitem, by fitting the mean and the two bias terms as described in the lecture notes. Use a regularization parameter of λ = 1. Report the MSE on the validation set (1 mark).

In [ ]:
from sklearn.metrics import mean_squared_error

# calculate alpha, the global average
allRatings = []
for user,book,r in readCSV("train_Interactions.csv.gz"):
    r = int(r)
    allRatings.append(r)
globalAverage = sum(allRatings) / len(allRatings)
alpha = globalAverage

In [ ]:
unique_user=list(set(l[0] for l in train_data))
unique_item=list(set(l[1] for l in train_data))

    
user_map = {}
for i, user in enumerate(unique_user):
    user_map[user] = i
item_map = {}
for i, item in enumerate(unique_item):
    item_map[item] = i

rate_table = np.zeros((len(unique_user), len(unique_item)))
for user,item,r in train_data:
    rate_table[user_map[user], item_map[item]] = r

user_items = defaultdict(list)
item_users = defaultdict(list)
for user, item, rating in train_data:
    user_items[user].append(item)
    item_users[item].append(user)

In [ ]:
def predict_one(I, a1, b1, b2):
    user,item = I[0],I[1]
    if user in user_map:
        if item in item_map:
            return a1 + b1[user_map[user]] + b2[item_map[item]]
    return alpha

In [ ]:
from sklearn.metrics import mean_squared_error
def train_parameter(a1, b1, b2, iters = 20, lambda_ = 1):
    mse = 0.0
    for iter_ in range(iters):
        a1_ = 0.0
        for I in train_data:
            a1_ += (int(I[2]) - b1[user_map[I[0]]] - b2[item_map[I[1]]])
        a1_ /= len(train_data)
        a1 = a1_
        for user_idx, user in enumerate(unique_user):
            b1_ = 0.0
            for item in user_items[user]:
                item_id = item_map[item]
                b1_ += (rate_table[user_idx,item_id] - b2[item_id] - a1)
            b1_ /= (lambda_ + len(user_items[user]))
            b1[user_idx] = b1_

        for item_idx, item in enumerate(unique_item):
            b2_ = 0.0
            for user in item_users[item]:
                user_id = user_map[user]
                b2_ += (rate_table[user_id,item_idx] - b1[user_id] - a1)
            b2_ /= (lambda_ + len(item_users[item]))
            b2[item_idx] = b2_
        mse = mean_squared_error([predict_one(I, a1, b1, b2) for I in valid_data], valid_ratings)
    return mse

In [ ]:
valid_ratings=allRatings[190000:]
b1 = np.random.rand(len(unique_user))
b2 = np.random.rand(len(unique_item))
a1 = alpha
mse = train_parameter(a1, b1, b2, iters = 20, lambda_ = 1)
print('Using Lambda:'+str(1)+' The MSR is: ' + str(mse))

#### 10. Report the user and book IDs that have the largest and smallest values of β (1 mark).

In [ ]:
index_min = np.argmin(b1)
index_max = np.argmax(b1)
print("user with the smallest B is: "+str(unique_user[index_min]))
print("user with the largest B is: "+str(unique_user[index_max]))
index_min = np.argmin(b2)
index_max = np.argmax(b2)
print("item with the smallest B is: "+str(unique_item[index_min]))
print("item with the largest B is: "+str(unique_item[index_max]))

In [ ]:
len(unique_user)

#### 11. Find a better value of λ using your validation set. Report the value you chose, its MSE, and upload your solution to Kaggle by running it on the test data (1 mark).

In [ ]:
las=np.arange(0.5, 4.5,0.5)
b1 = np.random.rand(len(unique_user))
b2 = np.random.rand(len(unique_item))
for i in range(0,len(las)):
    mse = train_parameter(a1, b1, b2, iters = 20, lambda_ = las[i])
    print('Using Lambda:'+str(las[i])+' The MSR is: ' + str(mse))

Using Lambda=3. The MSR is: 1.1082729198454861

In [ ]:
def train_parameter(a1, b1, b2, iters = 100, lambda_ = 3):
    mse = 0.0
    for iter_ in range(iters):
        a1_ = 0.0
        for I in train_data:
            a1_ += (int(I[2]) - b1[user_map[I[0]]] - b2[item_map[I[1]]])
        a1_ /= len(train_data)
        a1 = a1_
        for user_idx, user in enumerate(unique_user):
            b1_ = 0.0
            for item in user_items[user]:
                item_id = item_map[item]
                b1_ += (rate_table[user_idx,item_id] - b2[item_id] - a1)
            b1_ /= (lambda_ + len(user_items[user]))
            b1[user_idx] = b1_

        for item_idx, item in enumerate(unique_item):
            b2_ = 0.0
            for user in item_users[item]:
                user_id = user_map[user]
                b2_ += (rate_table[user_id,item_idx] - b1[user_id] - a1)
            b2_ /= (lambda_ + len(item_users[item]))
            b2[item_idx] = b2_
        mse = mean_squared_error([predict_one(I, a1, b1, b2) for I in valid_data], valid_ratings)
        print('At step' +str(iter_+1)+' The MSR is: ' + str(mse))
    return mse

In [ ]:
b1 = np.random.rand(len(unique_user))
b2 = np.random.rand(len(unique_item))
train_parameter(a1, b1, b2, iters = 100, lambda_ = 3)

In [ ]:
def train_parameter(a1, b1, b2, iters = 100, lambda_ = 3):
    mse = 0.0
    for iter_ in range(iters):
        a1_ = 0.0
        for I in train_data:
            a1_ += (int(I[2]) - b1[user_map[I[0]]] - b2[item_map[I[1]]])
        a1_ /= len(train_data)
        a1 = a1_
        for user_idx, user in enumerate(unique_user):
            b1_ = 0.0
            for item in user_items[user]:
                item_id = item_map[item]
                b1_ += (rate_table[user_idx,item_id] - b2[item_id] - a1)
            b1_ /= (lambda_ + len(user_items[user]))
            b1[user_idx] = b1_

        for item_idx, item in enumerate(unique_item):
            b2_ = 0.0
            for user in item_users[item]:
                user_id = user_map[user]
                b2_ += (rate_table[user_id,item_idx] - b1[user_id] - a1)
            b2_ /= (lambda_ + len(item_users[item]))
            b2[item_idx] = b2_
        mse = mean_squared_error([predict_one(I, a1, b1, b2) for I in valid_data], valid_ratings)
    return mse

b1 = np.random.rand(len(unique_user))
b2 = np.random.rand(len(unique_item))
mse=train_parameter(a1, b1, b2, iters = 30, lambda_ = 3)
print('Using Lambda 3 with '+str(30)+' iterations The MSR is: ' + str(mse))

In [ ]:
def predict_pair(user,item, a1, b1, b2): 
    if user in user_map:
        if item in item_map:
            return a1 + b1[user_map[user]] + b2[item_map[item]]
    return alpha

In [ ]:
predictions = open("predictions_Rating.txt", 'w')
for l in open("pairs_Rating.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
    continue
    user,item = l.strip().split('-')
    rating = predict_pair(user,item, a1, b1, b2)
    predictions.write(user + '-' + item + ',' + str(rating) + '\n')
predictions.close()